# generate unitary matrices for testing

In [15]:
import numpy as np

# this function generates a list of unitary matrices of dimension 2^n
# num specifies the number of matrices to be tested 
def generate_unitary_matrices(n, num, seed=None):
    if seed is not None:
        np.random.seed(seed)

    dim = 2 ** n
    unitary_matrices = []
    for _ in range(num):
        # Generate a random complex matrix of dimension 2^n x 2^n
        random_matrix = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
        # QR decomposition, Q is a unitary matrix
        Q, _ = np.linalg.qr(random_matrix)
        unitary_matrices.append(Q)
    return unitary_matrices

In [16]:
seed = 42

array1 = generate_unitary_matrices(1, 10, seed) # a list of ten 2^1 x 2^1 unitary matrices to be evaluated
array2 = generate_unitary_matrices(2, 10, seed)
array3 = generate_unitary_matrices(2, 10, seed)
array4 = generate_unitary_matrices(2, 10, seed)

In [ ]:
## a list of 5 matrices

# import pickle

# with open('array1.pkl', 'rb') as f:
#     array1 = pickle.load(f)

# with open('array2.pkl', 'rb') as f:
#     array2 = pickle.load(f)

# with open('array3.pkl', 'rb') as f:
#     array3 = pickle.load(f)

# with open('array4.pkl', 'rb') as f:
#     array4 = pickle.load(f)

# Evaluation of different methods
- Qiskit (For Qiskit decomposition, 2x2 unitary is just a U3-gate, so solovayKitaev in its library is used instead because it decomposes into basic gates like SGate, HGate, SdgGate. For larger unitaries, its module qiskit.transpile is used instead. The underlying algorithms this module used is mentioned in this post: https://quantumcomputing.stackexchange.com/questions/18157/how-efficient-is-qiskits-unitary-decomposition)
- OpenQL
- SQUANDER

In [ ]:
# import openQLs
import squander
import squander_test
import qiskit_decomposition
import solovayKitaev

In [19]:
import qiskit_decomposition
import solovayKitaev

## 1. 2x2 unitary evaluation

In [20]:
result1 = []
for arr in array1:
    # res = squander.squander_decomp(arr, 1)
    # res = qiskit_decomposition.qiskit_decomp(arr, 1)
    # res = openQLs.openQL_decomp(arr, 1)
    res = solovayKitaev.sk_decomp(arr, 1)
    # res = squander_test.decomp(arr,1)
    result1.append(res)

print(result1)


Original circuit:
   ┌───────────────┐
q: ┤ original gate ├
   └───────────────┘


NameError: name 'SdgGate' is not defined